In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
# config = Config(debug=False, sl=30, selected_columns=True, postpond=10)
config = Config(debug=False, sl=200, selected_columns=True, postpond=0)
# config = Config(debug=True, sl=200, selected_columns=True, postpond=0)
# config = Config(debug=True, sl=30, selected_columns=True, postpond=10)
torch.cuda.set_device(config.gpu_start); torch.ones((1, 1)).cuda(); # activate pytorch

In [ ]:
fns_feathers = sorted(config.fn_cycles.glob('cycle*'))[:config.num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
# if not config.selected_columns: config.cont_names = [o for o in cycles.loc[0].columns[2:] if o not in config.dep_var]
idx = fmtr.beginning_index(postpond=config.postpond)

In [ ]:
df_conts = tile_with_noise(cycles, idx, config, normalize=True)
deps = fmtr.get_y(config.dep_var)
deps = pd.concat([deps]*config.mulr).reset_index(drop=True)
cyc_cont = flatten_and_cat(df_conts, deps, sl=config.sl)
rnndb = MultiDeptTabularDataBunch.from_df('tmp', cyc_cont, config.dep_var, valid_idx=config.valid_idx_tile, bs=config.bs, num_workers=0)

Plot cycles to determine how to calculate y

In [ ]:
config.dep_var

In [ ]:
deps_full = [cycles.loc[o,config.dep_var] for o in cycles.index.levels[0]]
deps_out = fmtr.get_y(config.dep_var)

In [ ]:
for i, (curve, d1, d2) in enumerate(zip(deps_full, deps_out.values[:,0], deps_out.values[:,1])):
    if i % 10 == 0:
        ax = None
    ax = plots(curve, ax=ax, cols=2).flatten()
    ax[0].axhline(d1, color='r', linewidth='1', linestyle=':')
    ax[1].axhline(d2, color='r', linewidth='1', linestyle=':')